COMP4702 Report
================

Student: Hugo Burton

Date Due: Friday $24^{\text{th}}$ May 2024

In [1]:
import os
import sys
sys.path.append('..')

current_dir = os.getcwd()
print(current_dir)

from welcome import welcome, available_items
from check_log_level import set_log_level

set_log_level()
welcome()


C:\Users\burto\Documents\University (2021-2024)\2024 Semester 1\COMP4702\COMP4702\Project\report
Log level found LOG_LEVEL=DEBUG

Welcome to the Machine Learning Project
Torch version:  2.2.2+cpu
CUDA is not available
--------------------------------------------------
Python version: 3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]
Numpy version: 1.26.4
Seaborn version: 0.13.2
--------------------------------------------------


# Introduction

## Introduction to the dataaset

This report is an investigation into the quantitative traits of fruit flies, known scientifically as the species _Drosophila aldrichi_ and _D. buzzatii_. The data, collected in 1994 captures a range of traits of $n = 1731$ flies located on the East coast of Australia. The attached report on the dataset aims to contribute to understanding the factors driving latitudinal variation in size-related traits in the two species, considering both genetic adaptation and phenotypic plasticity in response to environmental conditions.

Fruit flies are a pest to many kinds of agriculture. Adult female flies lay their eggs within fruit and vegetable crops leading to their larvae hatching inside. These larvae then proceed to each the fruit from the inside out, causing rotting and thus destroying the crop. (Department of Agriculture, 2023)

## Preliminary Observations of the Datasets

Prior to determining an objective for this report, the dataset was analysed from a purely morphological perspective.

### Thorax and Wing Traits Dataset

A correlation plot of the `Thorax & wing traits` dataset is shown below. Evidently the columns species, population, temperature and sex all show a small magnitude of correlation with columns Thorax length through to wing_loading. 

Longitude and lattitude do show some correlation (other than with each other) - this could be interesting to investigate, though may not produce the best results from a classifier with this data alone.

The columns `Thorax_length` through to `wing_loading` show a highly positive correlation as seen in the bottom-right of the matrix, indicating these variables are very tightly related to each other. For example, we could say a fly with a higher than average Thorax length also very likely has a higher then average wing loading.

We note some columns as having no correlation with any other columns, namely `year_start` and `year_end`. Looking at the raw data, we see these are constant for all rows. In any analysis it would be a good decision to strip these from the model as they don't provide any additional information for classification purposes. Additionally, it was observed that there exist only distinct `Latitude` and `Longitudes` which are unique for each population. To make the classification fair, it would also make sense to remove these prior to training.

##### Thorax Correlation matrix

<img src="..\plots\Thorax_corr_matrix.png" width="700">



The Wing Traits and Asymmetry dataset does show correlation between each of the wing asymmetry traits, however, it shows little to no correlation between wing asymmetry and species and population. Meaning it is potentially not a good candidate dataset for making predictions about the fly's species and population.


#### Wing Traits and Asymmetry Correlation Matrix

<img src="..\plots\Wing_asymmetry_corr_matrix.png" width="700">


The `Wing Asymmetry`  dataset (which has different asymmetry traits from the previous dataset) does show signs of correlation between wing attributes and species & population, though this isn't as strong as the first dataset. It may, however, be a good candidate for determining temperature if that were of interest.

#### Wing Asymmetry Correlation Matrix

<img src="..\plots\Wing_traits_corr_matrix.png" width="700">


## Objective

While the report attached to these datasets (not this report) does provide explanation of what it is investigating, it does not provide detail on the reasoning behind doing so, other than simply to contribute to knowledge. This report will attempt to develop models which can classify both the **Species** and **Population** of fruit flies given the remaining variables collected. <!--While not covered in the attached report, knowing which species and population a fruit fly in its natural habitat belongs to may be helpful understanding its behaviour-->

Secondly, this report will attempt to perform feature selection by first ranking the input variables in descending order of importance. The top important variables will then be sliced from the dataset and used in a second round of classification for each model. Finally, comparisons can then be made between the models trained with all attributes, and models trained with the supposedly redundant attributes removed.

Finally, the report will determine a classifier which performs the best for the above object and discuss reasoning for any findings.

# Models

The objective defined above is related to classification of the Loeschcke dataset, meaning only classification models will be used. In this report, the following classifiers are:
- $k$-NN
- Decision Tree
- Neural Network


Where possible each model will be tested equally on the dataset. Note that in the case of a neural network, a single network can output multiple variables. However, this is not possible with $k$-NN or a decision tree, hence multiple classifiers will be instantiated and combined in order to make these predictions.


# Hardware

The models were trained on the following hardware

- Intel&reg; Core&trade; i7 12700K @ 5.0 GHz (8 P-cores, 4 E-cores)
- 32GB DDR5 @ 5200 MT/s
- Samsung 990 Pro 1TB
- Asus ROG STRIX GeForce RTX 3080-Ti OC @ 1940 MHz with 12GB GDDR6X @ 9630MHz
- Windows 11 Pro

CUDA compute was used to train the neural network model in a conda enviroment

## $k$-NN

### Model Setup

As two output variables are being predicted, two separate $k$-NN classifiers were constructed, each with equal parameter setups (e.g. train set, test set, cross validation method, etc).

### Selection of $k$ - Cross Validation

Cross validation was performed for each of the two $k$-NN classifiers (one for each output variable). A grid search over a wide range of $k$ values was performed in order to find the optimal value of $k$. To start, a $30\%$ test set was put aside and kept independent from cross validation. On the remaining $70\%$, each $k$ from $1$ through $100$ was trained on a randomly selected $\frac{4}{5}$ of the train set and validated on the remaining $\frac{1}{5}$. For each $k$, a new slicing into $5$ buckets was performed. The validation allowed us to obtain an accuracy value for each $k$ in the tested range. Once found, a final classifier was trained on a separate test set (not used in cross validation) to obtain the final results. A plot showing the accuracies for the output variable `Species` (from cross validation) of each $k$ is below:

<img src="..\plots\Thorax_knn_accuracies_Species.png" width="700">

The optimal value of $k$ found was $k = 9$. We observe the accuracy plot, though on a fine scale is not a smooth function. Partially this is due to the dataset being only $n = 1731$ rows which when sliced up into discrete bins used for training reduces the size of each effective training set significantly. Therefore, a small change in $k$ can result in a big change in accuracy due to being "lucky" or "unlucky" in classification being far more likely. Nonetheless, this does show a clear maximum on the lower end of the x-axis.

A plot showing the accuracies for the output variable `Population` is shown below. Note the same range of $k$ was tested.

<img src="..\plots\Thorax_knn_accuracies_Population.png" width="700">

We see from this graph that the optimal value of $k$ was much higher. This indicates population classes are likely more clustered than species. The reasoning behind this being that classes which are more clustered are less likely to decrease in performance with higher $k$ values due to having a majority of that class within a given region in hyperspace. Compared with if the classes are more overlapping or have scattered clusters, a change in $k$, even by $\Delta k = 1$ could significantly change the accuracy score, and we see this evident in the `Species` classifier.


## Decision Tree

The decision tree model had a similar setup to $k$-NN, however without cross validation being used. Due to the small number of features ($14$), the only hyperparameter of the max tree height was chosen at $h_{\text{max}} = 10$. Instead, the decision tree was utilised for determining the importance of variables or predictors in the dataset. This was done independently for each of the two output variables.

Hence when training both the $k$-NN and neural network models, the decision tree was trained on the same data first, not for it's performance, but instead to obtain a ranking of predictor variables. This was then used in order to determine which decision boundary plots to show, given that plotting all of them would be infeasible and difficult to understand.

## Neural Network



# Ranking of Predictor Variables



# Decision Boundaries

During training of each classifier a total of $14$ predictor variables were fed as inputs for training. While this is by no means a high number of input variables, it would be impractical to plot every pairwise combination of the features as a decision boundary as there would simply be too many plots to show and analyse. We note that decision boundary plots can only really be shown in 2 dimensions (at least for now or without AR!). Instead, using the technique described in the **Ranking of Predictor Variables** section, only the top $\delta = 5$ variables were considered for their decision boundary. We are interested in the top ranking predictor variables because they are the ones most likely to show a manifold like structure of the data.

When plotting pairs of variables, the remaining features are held constant at their respective mean. This does have limitations in that some plots may misrepresent the boundaries, particularly if the size of each class isn't equal, however, it allows us to visualise the model in an easy to understand way.

I would argue holding the non-plotted variables at their mean is superior to ploting e.g. PCA because the former method does not cause loss of interpretability of the data. From the plots below we can clearly see the regions of the classes learned by the model with respect to specific variables. Using a dimensionality reduction technique does not make sense here given the already small number of features.

In the case of hundreds or thousands of input variables where many of them turn to be important, then PCA could be considered usedful, however, it was not used in this report for the reasons specified above.

## Decision Boundaries of Species

Firstly, we will consider only the decision boundaries of the `Species` output variable.

### $k$-NN

<img src="..\plots\knn_Thorax_decision_boundaries_Species.png" width="700">

It is evident from the plot above that `Thorax Length` and `Wing Loading` explain the classes the best. Almost amazingly, we see the plot of the combination of these variables (top-right) as showing almost no wrong predictions.

The four plots closest to the bottom show that `w_2` and `Temperature` are not able to predict species very well at all. They only appear at the top of the predictor variables ranking because there are simply no other variables (other than `Thorax_length` and `Wing loading`) which are comparable.


### Decision Tree

<img src="..\plots\dt_Thorax_decision_boundaries_Species.png" width="700">

The decision tree did not perform as well as $k$-NN, achiving only ___% compared with ___% test accuracy for $k$-NN. The decision tree plot above highlights this finding. Observe (at least for the top predictors) the data is highly correlated - something which a decision tree does not perform well at due to it's lack of ability to draw decision boundaries that are not orthogonal to the axes. 

We can see in `Wing loading` vs `w2`, the `D._aldrichi` species is almost clustered in the middle of the `D._buzzatii` species. The decision tree did create a small red area in the top in an attempt to capture this, however, it is evident the border is not optimal when compared with the other classifiers.

We note the decision tree really does not perform well with the top-two classsifiers `Thorax length` and `wing loading` due to the scattered nature of the classes. This is very different from the $k$-NN classifier where these two predictors were the most critical to enabling the model to perform well.


### Neural Network


<img src="..\plots\nn_Thorax_decision_boundaries_Species.png" width="700">


## Decision Boundaries of Population

Secondly, we will consider only the decision boundaries of the `Population` output variable.

### $k$-NN

<img src="..\plots\knn_Thorax_decision_boundaries_Population.png" width="700">


### Decision Tree


Highly correlated data - not great for decision tree.


<img src="..\plots\dt_Thorax_decision_boundaries_Population.png" width="700">

### Neural Network


<img src="..\plots\nn_Thorax_decision_boundaries_Population.png" width="700">



## Certainty of Classification

# Ranking of Classifier Models

The best classifications occur when the certainty is high and the prediction of class is correct. One sample from the
test set was chosen from each classifier for each true class which had maximum certainty and the prediction was
correct. The results are outlined below.

# Worst Errors


# Conclusion

# References (yet to compile)

1. https://www.agriculture.gov.au/biosecurity-trade/pests-diseases-weeds/fruit-flies-australia

# Appendix